In [165]:
import requests
import json
from pathlib import Path
import pandas as pd
import sqlite3 as sql
import os

# Coleta e Transformação de Dados do IPCA

In [150]:
dados = pd.DataFrame(pd.read_json("https://servicodados.ibge.gov.br/api/v3/agregados/1420/periodos/201201%7C201202%7C201203%7C201204%7C201205%7C201206%7C201207%7C201208%7C201209%7C201210%7C201211%7C201212%7C201301%7C201302%7C201303%7C201304%7C201305%7C201306%7C201307%7C201308%7C201309%7C201310%7C201311%7C201312%7C201401%7C201402%7C201403%7C201404%7C201405%7C201406%7C201407%7C201408%7C201409%7C201410%7C201411%7C201412%7C201501%7C201502%7C201503%7C201504%7C201505%7C201506%7C201507%7C201508%7C201509%7C201510%7C201511%7C201512%7C201601%7C201602%7C201603%7C201604%7C201605%7C201606%7C201607%7C201608%7C201609%7C201610%7C201611%7C201612%7C201701%7C201702%7C201703%7C201704%7C201705%7C201706%7C201707%7C201708%7C201709%7C201710%7C201711%7C201712%7C201801%7C201802%7C201803%7C201804%7C201805%7C201806%7C201807%7C201808%7C201809%7C201810%7C201811%7C201812%7C201901%7C201902%7C201903%7C201904%7C201905%7C201906%7C201907%7C201908%7C201909%7C201910%7C201911%7C201912/variaveis/306?localidades=N1%5Ball%5D&classificacao=315%5B7169%5D"))
dados = dados['resultados'][0][0]['series'][0]['serie']

listaMesDados = []
listaAnoDados = []
listaIPCADados = []
for data in dados:
    listaMesDados.append(data[4:6])
    listaAnoDados.append(data[0:4])
    listaIPCADados.append(dados[data])

listaDados = {'IPCA':listaIPCADados, 'Mês':listaMesDados, 'Ano':listaAnoDados}

dados = pd.DataFrame(listaDados)
dados

,IPCA,Mês,Ano
0,0.48,01,2012
1,0.48,02,2012
2,0.19,03,2012
3,0.63,04,2012
4,0.39,05,2012
...,...,...,...
91,0.19,08,2019
92,-0.01,09,2019
93,0.07,10,2019
94,0.44,11,2019


# Criação do Banco de Dados

In [164]:
endereco = Path(os.getcwd() + '\\Bancos de Dados\\')

BDDW = endereco / 'IPCADW.db'

if endereco.exists():
    print('Endereço existe: ', endereco)
    if BDDW.exists():
        print('Banco de dados já existe.')
    else:
        BDDW.touch()
        print('Banco de dados acabou de ser criado.')
else:
    print('Endereço não existe. Verificar o caminho.')

Endereço existe:  C:\Users\elias\Downloads\Bancos de Dados
Banco de dados já existe.


# Conexão com o Banco de Dados

In [170]:
# conexão com o banco de dados
conexaoDW = sql.connect(BDDW)
# definição do cursor
sqlDW = conexaoDW.cursor()

# Definição da Tabela no Banco de Dados (DDL)

In [185]:
# definição das querys
queryTabela = '''
    CREATE TABLE IF NOT EXISTS tbDadosIPCA (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        IPCA REAL(2,2) NOT NULL,
        mes VARCHAR(2) NOT NULL,
        ano VARCHAR(4) NOT NULL  
    )
'''
queryIndice = 'CREATE INDEX IF NOT EXISTS idx_tbDadosIPCA_id ON tbDadosIPCA(id)'

# execução das querys
sqlDW.execute(queryTabela)
sqlDW.execute(queryIndice)


# Carga do DF ao Banco de Dados (DML)

In [186]:
print('Iniciando a carga de dados')

sqlDW.executemany('''INSERT INTO tbDadosIPCA (IPCA, mes, ano) VALUES (?,?,?)''', dados.values.tolist())
conexaoDW.commit()

print('Carga de', len(dados), 'registros em tbDadosIPCA finalizada')

Iniciando a carga de dados
Carga de 96 registros em tbDadosIPCA finalizada


# Validação da Carga de Dados (DQL)

In [187]:
pd.read_sql('SELECT * FROM tbDadosIPCA', conexaoDW)

,id,IPCA,mes,ano
0,1,0.48,01,2012
1,2,0.48,02,2012
2,3,0.19,03,2012
3,4,0.63,04,2012
4,5,0.39,05,2012
...,...,...,...,...
91,92,0.19,08,2019
92,93,-0.01,09,2019
93,94,0.07,10,2019
94,95,0.44,11,2019


# Estudo de Cenários

In [221]:
# Médias anuais do IPCA na série histórica
pd.read_sql('''SELECT round(avg(IPCA),2) as "Média IPCA", ano as "Ano" FROM tbDadosIPCA GROUP BY ano''',conexaoDW)

,Média IPCA,Ano
0,0.48,2012
1,0.48,2013
2,0.52,2014
3,0.85,2015
4,0.51,2016
5,0.24,2017
6,0.31,2018
7,0.35,2019


In [223]:
# Ano de menor variação no IPCA da série histórica
pd.read_sql('''SELECT MIN("Média IPCA") as "Média IPCA", "Ano" FROM (
SELECT round(avg(IPCA),2) as "Média IPCA", ano as "Ano" FROM tbDadosIPCA GROUP BY ano ORDER BY avg(IPCA) ASC)''',conexaoDW)

,Média IPCA,Ano
0,0.24,2017


In [230]:
# Mês com maior aumento do IPCA
pd.read_sql('''SELECT max(IPCA) as "IPCA", mes as "Mês", ano as "Ano" FROM tbDadosIPCA''',conexaoDW)

,max(IPCA),Mês,Ano
0,1.35,06,2018
